In [ ]:
from promptx import prompt
from datasets import load_dataset

In [ ]:
data = load_dataset('gsm8k', 'main')

In [ ]:
train = data['train']
train['question'][:5]

In [ ]:
examples = [
    (
        {
            'question': "Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?",
            'answer': "Julie read 12 x 2 = <<12*2=24>>24 pages today. So she was able to read a total of 12 + 24 = <<12+24=36>>36 pages since yesterday. There are 120 - 36 = <<120-36=84>>84 pages left to be read. Since she wants to read half of the remaining pages tomorrow, then she should read 84/2 = <<84/2=42>>42 pages. #### 42"
        },
        '''
        Let \( T \) be the total number of pages in the book, \( P_y \) be the number of pages read yesterday, and \( P_t \) be the number of pages read today. The equation to find the number of pages \( P_r \) to be read tomorrow is given by:

        \[ P_r = \frac{1}{2} \left( T - P_y - P_t \right) \]

        This equation states that the number of pages to be read tomorrow (\( P_r \)) is half of the remaining pages after subtracting the pages read yesterday (\( P_y \)) and today (\( P_t \)) from the total number of pages in the book (\( T \)).

        Given:
        - \( T = 120 \) pages (total pages in the book)
        - \( P_y = 12 \) pages (pages read yesterday)
        - \( P_t = 2 \times P_y = 24 \) pages (pages read today, which is twice the pages read yesterday)

        Applying the values to the equation:

        \[ P_r = \frac{1}{2} \left( 120 - 12 - 24 \right) \]
        \[ P_r = \frac{1}{2} \left( 120 - 36 \right) \]
        \[ P_r = \frac{1}{2} \times 84 \]
        \[ P_r = 42 \]

        Therefore, Julie should read 42 pages tomorrow to read half of the remaining pages.
        ####
        42
        ''',
    ),
    (
        {
            'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
            'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May. Natalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May. #### 72',
        },
        '''
        Let \( C_A \) represent the number of clips sold in April, and \( C_M \) represent the number of clips sold in May. The total number of clips \( C_T \) sold in April and May is given by:

        \[ C_T = C_A + C_M \]

        Since \( C_M \) is half the number of clips sold in April, we can express \( C_M \) as \( \frac{1}{2} C_A \). Therefore, the equation becomes:

        \[ C_T = C_A + \frac{1}{2} C_A \]
        \[ C_T = \frac{3}{2} C_A \]

        This equation states that the total number of clips sold in April and May (\( C_T \)) is one and a half times the number of clips sold in April (\( C_A \)).

        Given:
        - \( C_A = 48 \) clips (clips sold in April)

        Applying the values to the equation:

        \[ C_T = \frac{3}{2} \times 48 \]
        \[ C_T = 1.5 \times 48 \]
        \[ C_T = 72 \]

        Therefore, Natalia sold a total of 72 clips in April and May combined.
        ####
        72
        '''
    ),
]

sample = train[-5:]
rs = []
for q, a in zip(sample['question'], sample['answer']):
    r = prompt(
        '''
        Given a question and an answer, write a formula to solve the general case using algebraic notation
        ''',
        input={
            'question': q,
            'answer': a,
        },
        examples=examples,
    )
    rs.append(r)

rs

In [ ]:
import json

sample = train[180:]
rs = []
file_counter = 18
row_counter = 0
save_every_n_rows = 10

for q, a in zip(sample['question'], sample['answer']):
    r = prompt(
        '''
        Given a question and an answer, write a formula to solve the general case using algebraic notation.
        Ensure you always end your answer with #### followed by the answer as a single numerical value. E.g. 

        ####
        42
        ''',
        input={
            'question': q,
            'answer': a,
        },
        examples=examples,
    )
    # Structure the result as a dictionary with question and generated answer
    result = {'question': q, 'answer': r}
    rs.append(result)
    row_counter += 1

    # Check if the row counter has reached 1000
    if row_counter == save_every_n_rows:
        # Save the current results to a JSONL file
        with open(f'results_{file_counter}.jsonl', 'w') as f:
            for item in rs:
                f.write(json.dumps(item) + '\n')

        # Reset the row counter and results list, increment the file counter
        row_counter = 0
        rs = []
        file_counter += 1

# Don't forget to save the remaining results if there are any
if rs:
    with open(f'results_{file_counter}.jsonl', 'w') as f:
        for item in rs:
            f.write(json.dumps(item) + '\n')
